In [54]:
# The cache directories:
cache_path       = '/data/crcns2021'
image_cache_path = f'{cache_path}/annot-images'
v123_cache_path  = f'{cache_path}/annot-v123'
csulc_cache_path = f'{cache_path}/annot-csulc'
trace_save_path = f'{cache_path}/results/data_branch/save'
labels_path = f'{cache_path}/results/labels'

import os
# If you aren't using /data
os.environ['HCPANNOT_LOAD_PATH'] = cache_path

In [55]:
import os 
import sys
sys.path.append('..')
import numpy as np
import neuropythy as ny
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
import matplotlib as mpl
# Additional matplotlib preferences:
font_data = {'size': 10,
             'weight': 'light',
             'family':'sans-serif',
             'sans-serif':['Helvetica']} # missing fonts 
mpl.rc('font', **font_data) 
mpl.rc('text', usetex=True)
# we want relatively high-res images, especially when saving to disk.
mpl.rcParams['figure.dpi'] = 72 * 2
mpl.rcParams['savefig.dpi'] = 72 * 4

SyntaxError: positional argument follows keyword argument unpacking (289588141.py, line 7)

In [57]:
# Now we can import the hcp-annot-vc library.
import hcpannot
hcpannot.interface.default_load_path = cache_path  
from hcpannot import (vc_plan, 
                      save_contours, 
                      load_contours)


In [58]:
rater1 = 'BrendaQiu'
rater2 = 'JiyeongHa'
sid1 = 111312
sid2 = 102311

## Helper functions for plotting mean contours across subjects

In [70]:
def roi_trace_to_path(dat, subject, roi, hemi):
    # grab a trace
    trace = dat['traces'][roi]
    # get the subject's hemisphere mesh
    hemi_surf = ny.data['hcp_lines'].subjects[subject].hemis[hemi]
    path = trace.to_path(hemi_surf)
    return path
    
def get_path_coords_in_fsaverage(path, hemi):
    fmap = ny.to_flatmap('occipital_pole', hemi)
    fsa_coords = fmap.unaddress(path.addresses)
    return fsa_coords

def _normalize_coords(fsa_coords, n_points=800):
    curve = ny.util.CurveSpline(fsa_coords)
    normed_fsa_coords = curve.linspace(n_points)
    return normed_fsa_coords

def normalize_multiple_coords(fsa_coords_list, fsa_coords_key_list=None, n_points=800, avgerage=True):
    if fsa_coords_key_list is None:
        fsa_coords_key_list = np.range(0, len(fsa_coords_list))
    tmp = []
    for fsa_coords in fsa_coords_list:
        tmp.append(_normalize_coords(fsa_coords, n_points))
    normed_fsa_coords = dict(zip(fsa_coords_key_list, tmp))
    
    if average is True:
        normed_fsa_coords['avg'] = np.mean(tmp, axis=0)
    return normed_fsa_coords

def plot_mean_contours_across_subjects(rater, roi, subject_list, hemi, n_points=800, trace_save_path=None):
    fsa_coords_list = []
    for sid in subject_list:
        dat = vc_plan(rater=rater, sid=sid, hemisphere=hemi, save_path=trace_save_path)
        path = roi_trace_to_path(dat, sid, roi, hemi)
        fsa_coords += get_path_coords_in_fsaverage(path, hemi)
    normed_fsa_coords_list = normalize_multiple_coords(fsa_coords_list, 
                                                       fsa_coords_key_list=subject_list, 
                                                       n_points, 
                                                       avgerage=True)
    return normed_fsa_coords_list


## Plot functions

In [ ]:
def plot_multiple_traces(normed_fsa_coords):
    fig, ax = plt.subplots(1,1)
    for k, fsa_coords in normed_fsa_coords.items():
        (x,y) = fsa_coords
        ax.plot(x,y, label=k)
    plt.legend(loc='best')

## for savings..

In [82]:
ny.save(f'{sid1}_{rater1}_fsacoords.mgz', fsa_normalized_coords1)
    